In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
df = pd.read_csv('cleanedData.csv', low_memory=False)

In [3]:
df = df.drop(columns=['Unnamed: 0'], errors='ignore')
df.sample(3)

,Body Type,Driven(KM),No of Owner,Brand(Model),Variant Name,Price(Lakhs),City,Fuel Type,RTO,Transmission,...,Steering Type,Turning Radius,Front Brake Type,Rear Brake Type,Top Speed(KMPH),Acceleration,No Door Numbers,Cargo Volumn,Wheel Size,Age(Months)
4011,SUV,61200.0,1,Land Rover Range Rover Sport,HSE,63.00,Delhi,Diesel,UK15,Automatic,...,Power,6.05,Ventilated Disc,Ventilated Disc,210.0,7.2,5.0,784.0,20.0,8.0
1278,Hatchback,21000.0,1,Maruti Wagon R,VXI AMT,5.39,Bangalore,Petrol,KA05,Automatic,...,Power,4.70,Disc,Drum,NaN,18.6,5.0,341.0,NaN,7.0
4095,SUV,24788.0,1,MG Astor,Savvy Turbo AT BSVI,15.41,Delhi,Petrol,DL12,Automatic,...,Electronic,NaN,Disc,Disc,NaN,NaN,5.0,NaN,17.0,3.0


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [39]:
X = df.drop(columns=['Price(Lakhs)'])
y = df['Price(Lakhs)']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

((6695, 54), (1674, 54))

In [41]:
# The columns with more than 80% same values has no proper contribution to the model.
# So, the columns have to be dropped

X_train_same_value_threshold = 0.8
X_train_same_value_drop_list = [col for col in X_train.columns if X_train[col].value_counts(normalize=True).values[0] > X_train_same_value_threshold]
X_train_same_value_drop_list

['Steering',
 'Front',
 'Lights',
 'Rear',
 'Opener',
 'Beams',
 'Bag',
 'Seats',
 'Drive Type']

In [42]:
X_train = X_train.drop(columns=X_train_same_value_drop_list, axis=1)
X_test = X_test.drop(columns=X_train_same_value_drop_list, axis=1)

In [43]:
# The columns with more than 50% missing values have no proper contribution to the model.
# So, the columns have to be dropped

X_train_missing_value_threshold = 0.5
X_train_missing_value_drop_list = [col for col in X_train.columns if X_train[col].isna().mean() > X_train_missing_value_threshold]
X_train_missing_value_drop_list

[]

In [44]:
# IMPUTING MISSING VALUES

In [45]:
num_col_list = X_train.select_dtypes(include=['float64']).columns.to_list()

In [46]:
# Filling missing values with the KNNImputing method
kimputer = KNNImputer(weights='distance')

X_train[num_col_list] = kimputer.fit_transform(X_train[num_col_list])
X_test[num_col_list] = kimputer.transform(X_test[num_col_list])

In [47]:
cat_col_list = X_train.select_dtypes(include=['object']).columns.to_list()

In [48]:
for col in cat_col_list:
    X_train.loc[X_train[col].isna(), col] = X_train[col].dropna().sample(X_train[col].isna().sum()).values
    X_test.loc[X_test[col].isna(), col] = X_train[col].dropna().sample(X_test[col].isna().sum()).values

In [49]:
# DETECTING AND CAPPING OUTLIERS

In [50]:
for col in num_col_list:

    # Quantiles range calculation
    Q1 = X_train[col].quantile(0.25)
    Q3 = X_train[col].quantile(0.75)

    # Inter Quartile Range calculation
    IQR = Q3 - Q1

    # Minimum and Maximum values calculations for outliers
    lower_value = Q1 - 1.5 * IQR
    upper_value = Q3 + 1.5 * IQR

    # Capping outliers with the max and min values for Training data
    X_train.loc[X_train[col] > upper_value, col] = upper_value
    X_train.loc[X_train[col] < lower_value, col] = lower_value

    # Capping outliers with the max and min values for Testing data
    X_test.loc[X_test[col] > upper_value, col] = upper_value
    X_test.loc[X_test[col] < lower_value, col] = lower_value

In [51]:
# CORRELATION ANALYSIS

In [52]:
corr_matrix = pd.concat([X_train[num_col_list], y_train], axis=1).corr().abs()
threshold = 0.8

# Find columns that are highly correlated
high_corr_pairs = np.where(corr_matrix > threshold)
high_corr_pairs = [(corr_matrix.index[x], corr_matrix.columns[y]) for x, y in zip(*high_corr_pairs) if x != y and x < y]
high_corr_pairs

[('Max Power', 'Torque'),
 ('Max Power', 'Displacement'),
 ('Max Power', 'Length(mm)'),
 ('Max Power', 'Width(mm)'),
 ('Max Power', 'Wheel Base(mm)'),
 ('Max Power', 'Kerb Weight(KG)'),
 ('Max Power', 'Turning Radius'),
 ('Max Power', 'Top Speed(KMPH)'),
 ('Max Power', 'Acceleration'),
 ('Max Power', 'Wheel Size'),
 ('Torque', 'Displacement'),
 ('Torque', 'Kerb Weight(KG)'),
 ('Displacement', 'Kerb Weight(KG)'),
 ('Displacement', 'Turning Radius'),
 ('Length(mm)', 'Width(mm)'),
 ('Length(mm)', 'Wheel Base(mm)'),
 ('Length(mm)', 'Turning Radius'),
 ('Width(mm)', 'Wheel Base(mm)'),
 ('Width(mm)', 'Front Tread'),
 ('Width(mm)', 'Rear Tread'),
 ('Width(mm)', 'Wheel Size'),
 ('Wheel Base(mm)', 'Turning Radius'),
 ('Front Tread', 'Rear Tread'),
 ('Kerb Weight(KG)', 'Turning Radius'),
 ('Top Speed(KMPH)', 'Acceleration')]

In [53]:
# By observing the above highly correlating columns have to be dropped.
# No of Cylinder and No Door Number columns not relating to the Target column. So, the columns have to be dropped. 

cols_drop_list = ['Turning Radius', 'Kerb Weight(KG)', 'Front Tread', 'Rear Tread', 'Wheel Size', 
                  'No Door Numbers', 'No of Cylinder', 'Wheel Base(mm)']
X_train = X_train.drop(columns = cols_drop_list, axis=1)
X_test = X_test.drop(columns = cols_drop_list, axis=1)

In [54]:
X_train['Area'] = 2 * (X_train['Length(mm)'] + X_train['Width(mm)'])
X_test['Area'] = 2 * (X_test['Length(mm)'] + X_test['Width(mm)'])

X_train = X_train.drop(columns = ['Length(mm)', 'Width(mm)'], axis=1)
X_test = X_test.drop(columns = ['Length(mm)', 'Width(mm)'], axis=1)

In [55]:
cat_cols_drop_list = ['Front Brake Type', 'Rear Brake Type']

In [56]:
# By domain knowledge, the above categorical columns will not affect the price much. So, the columns have to be dropped.

X_train = X_train.drop(columns = cat_cols_drop_list, axis=1)
X_test = X_test.drop(columns = cat_cols_drop_list, axis=1)

In [57]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [58]:
# SCALING THE NUMERICAL COLUMNS

In [59]:
scaling_list = X_train.select_dtypes(include=['float64']).columns.to_list()

scaler = StandardScaler()
X_train[scaling_list] = scaler.fit_transform(X_train[scaling_list])
X_test[scaling_list] = scaler.transform(X_test[scaling_list])

In [60]:
# ENCODING THE CATEGORICAL COLUMNS

In [61]:
target_en_cols = ['Brand(Model)', 'Variant Name', 'RTO', 'Color', 'Engine Type', 'Fuel Suppy System']
for col in target_en_cols:
    mean_target = df.groupby(col)['Price(Lakhs)'].mean()
    X_train[col] = X_train[col].map(mean_target)
    X_test[col] = X_test[col].map(mean_target)

In [62]:
ohe_en_cols = ['Body Type', 'Fuel Type', 'Transmission', 'City', 'Value Configuration', 'Light', 'Headrest', 'Antenna', 'Headlamps', 'System', 
               'Wheel', 'Control', 'Warning', 'Turbo Charger', 'Steering Type']

In [63]:
ohencoder = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)

X_train_encoded = ohencoder.fit_transform(X_train[ohe_en_cols])
X_test_encoded = ohencoder.transform(X_test[ohe_en_cols])

encoded_columns = ohencoder.get_feature_names_out(ohe_en_cols)

X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoded_columns, index=X_train.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoded_columns, index=X_test.index)

X_train = pd.concat([X_train.drop(ohe_en_cols, axis=1), X_train_encoded_df], axis=1)
X_test = pd.concat([X_test.drop(ohe_en_cols, axis=1), X_test_encoded_df], axis=1)

C:\Users\HP\Documents\Python\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0, 4, 13, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [70]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [77]:
base_learners = [('lr3', LinearRegression()),
                ('svr', SVR()),
                ('dt', DecisionTreeRegressor(max_depth=10, min_samples_split=4, random_state = 42)),
                ('rf', RandomForestRegressor(n_estimators = 150, max_depth=10, min_samples_leaf=4, random_state = 42))]
meta_learner = LinearRegression()
stacking_model = StackingRegressor(estimators = base_learners, final_estimator = meta_learner, cv=5)
stacking_model.fit(X_train, y_train)

StackingRegressor(cv=5,
                  estimators=[('lr3', LinearRegression()), ('svr', SVR()),
                              ('dt',
                               DecisionTreeRegressor(max_depth=10,
                                                     min_samples_split=4,
                                                     random_state=42)),
                              ('rf',
                               RandomForestRegressor(max_depth=10,
                                                     min_samples_leaf=4,
                                                     min_samples_split=10,
                                                     n_estimators=150,
                                                     random_state=42))],
                  final_estimator=LinearRegression())

In [78]:
y_pred = stacking_model.predict(X_test)
y_train_pred = stacking_model.predict(X_train)

In [79]:
print(r2_score(y_test, y_pred))
print(r2_score(y_train, y_train_pred))

0.8960796303909798
0.9566497274483557


In [68]:
cat_cols = ['Light', 'Headrest', 'Antenna', 'Headlamps', 'System', 'Variant Name', 'RTO', 'Engine Type', 'Fuel Suppy System', 
            'Wheel', 'Control', 'Warning', 'Turbo Charger', 'Steering Type', 'Color', 'Value Configuration', 'Body Type', 'City']
num_cols = ['Mileage', 'Torque', 'Displacement', 'Acceleration', 'Cargo Volumn', 'No of Owner', 'Top Speed', 'Height']

In [69]:
cat_pattern = '|'.join(cat_cols)
num_pattern = '|'.join(num_cols)

X_train = X_train.drop(X_train.filter(regex=cat_pattern).columns, axis=1)
X_test = X_test.drop(X_test.filter(regex=cat_pattern).columns, axis=1)

X_train = X_train.drop(X_train.filter(regex=num_pattern).columns, axis=1)
X_test = X_test.drop(X_test.filter(regex=num_pattern).columns, axis=1)